In [47]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

# Non-DataScience/ML packages
import chardet
import re
import string

In [48]:
path="../../../../Datafiles/"
file = 'table_8_offenses_known_to_law_enforcement_new_york_by_city_2013.xls'
df = pd.read_excel(path + file, header=4)
print("there are {} entries in the data frame".format(len(df)))

there are 351 entries in the data frame


In [49]:
# Let's clean up the column names
colnames = df.columns
newcolnameslist = []

for colname in colnames:
    newcolname=colname.replace('\n','').replace(' ','').capitalize() # strip out newlines, spaces, and captitalize
    newcolname=re.sub(r'\([^)]*\)', '', newcolname)                  # remove parenthesized stuff
    newcolnameslist.append(newcolname)                               # put all the column names into a list 

df.columns = newcolnameslist
print(newcolnameslist)

['City', 'Population', 'Violentcrime', 'Murderandnonnegligentmanslaughter', 'Rape1', 'Rape2', 'Robbery', 'Aggravatedassault', 'Propertycrime', 'Burglary', 'Larceny-theft', 'Motorvehicletheft', 'Arson3']


In [50]:
df.tail(5)

,City,Population,Violentcrime,Murderandnonnegligentmanslaughter,Rape1,Rape2,Robbery,Aggravatedassault,Propertycrime,Burglary,Larceny-theft,Motorvehicletheft,Arson3
346,Yonkers,199134.0,1036.0,6.0,NaN,25.0,390.0,615.0,2368.0,470.0,1662.0,236.0,10.0
347,Yorktown Town,36643.0,15.0,0.0,NaN,0.0,2.0,13.0,334.0,45.0,287.0,2.0,NaN
348,1 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,2 The figures shown in this column for the off...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,3 The FBI does not publish arson data unless i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
df[df['City'].str.contains('figures shown')]
df[df['City'].str.contains('publish')]

df = df[(df['City'].str.contains('figures shown') == False)] # We don't want any documentation rows
df = df[(df['City'].str.contains('publish') == False)]       # We don't want any other documentation rows
# All Rape1 values are false
# No Rape2 values are true
# Arson3 NaN's -> 161
# There are 161 NaN entries for Arson3

In [52]:
df['PopulationSquared'] = df.Population.pow(2)

df['Robbery'] = np.where(df['Robbery']>=1, 1, 0.0)
df = df.rename(columns={'Arson3': 'Arson', 'Rape2': 'Rape', 'Aggravatedassault': 'AggravatedAssault',
                        'Violentcrime':'ViolentCrime','Propertycrime': 'PropertyCrime', 
                        'Larceny-theft':'LarcenyTheft','Motorvehicletheft':'MotorVehicleTheft'})
# df.Arson = df.Arson.astype(int) # Let's make it an int
df['Murder'] = np.where(df['Murderandnonnegligentmanslaughter']>=1, 1, 0.0)
df.Arson.fillna(0, inplace=True) # Let's clean up Arson3
df['Murder'] = df['Murder'].astype(int)
df['Population'] = df['Population'].astype(int)
df['ViolentCrime'] = df['ViolentCrime'].astype(int)
df['Rape'] = df['Rape'].astype(int)
df['Robbery'] = df['Robbery'].astype(int)
df['AggravatedAssault'] = df['AggravatedAssault'].astype(int)
df['PropertyCrime'] = df['PropertyCrime'].astype(int)
df['Burglary'] = df['Burglary'].astype(int)
df['LarcenyTheft'] = df['Murder'].astype(int)
df['MotorVehicleTheft'] = df['Murder'].astype(int)

In [53]:
df.head(50)
# df.describe()

,City,Population,ViolentCrime,Murderandnonnegligentmanslaughter,Rape1,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,PopulationSquared,Murder
0,Adams Village,1861,0,0.0,NaN,0,0,0,12,2,0,0,0.0,3.463321e+06,0
1,Addison Town and Village,2577,3,0.0,NaN,0,0,3,24,3,0,0,0.0,6.640929e+06,0
2,Akron Village,2846,3,0.0,NaN,0,0,3,16,1,0,0,0.0,8.099716e+06,0
3,Albany,97956,791,8.0,NaN,30,1,526,4090,705,1,1,0.0,9.595378e+09,1
4,Albion Village,6388,23,0.0,NaN,3,1,16,223,53,0,0,0.0,4.080654e+07,0
5,Alfred Village,4089,5,0.0,NaN,0,1,2,46,10,0,0,0.0,1.671992e+07,0
6,Allegany Village,1781,3,0.0,NaN,0,0,3,10,0,0,0,0.0,3.171961e+06,0
7,Amherst Town,118296,107,1.0,NaN,7,1,68,2118,204,1,1,3.0,1.399394e+10,1
8,Amityville Village,9519,9,0.0,NaN,2,1,3,210,16,0,0,1.0,9.061136e+07,0
9,Amsterdam,18182,30,0.0,NaN,0,1,18,405,99,0,0,0.0,3.305851e+08,0


In [54]:
df.describe()

,Population,ViolentCrime,Murderandnonnegligentmanslaughter,Rape1,Rape,Robbery,AggravatedAssault,PropertyCrime,Burglary,LarcenyTheft,MotorVehicleTheft,Arson,PopulationSquared,Murder
count,3.480000e+02,348.000000,348.000000,0.0,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,348.000000,3.480000e+02,348.000000
mean,4.003763e+04,201.594828,1.566092,NaN,5.864943,0.600575,121.261494,792.606322,119.683908,0.140805,0.140805,1.005747,2.035547e+11,0.140805
std,4.500374e+05,2815.268504,18.303673,NaN,60.425452,0.490486,1706.131730,7659.724746,924.948789,0.348321,0.348321,7.884612,3.778876e+12,0.348321
min,5.260000e+02,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.766760e+05,0.000000
25%,3.003000e+03,2.000000,0.000000,NaN,0.000000,0.000000,1.000000,40.500000,6.000000,0.000000,0.000000,0.000000,9.018117e+06,0.000000
50%,7.233500e+03,6.000000,0.000000,NaN,0.000000,1.000000,4.000000,112.500000,17.500000,0.000000,0.000000,0.000000,5.232568e+07,0.000000
75%,1.842750e+04,22.000000,0.000000,NaN,2.000000,1.000000,14.000000,341.000000,51.250000,0.000000,0.000000,0.000000,3.397536e+08,0.000000
max,8.396126e+06,52384.000000,335.000000,NaN,1112.000000,1.000000,31767.000000,141971.000000,16606.000000,1.000000,1.000000,132.000000,7.049493e+13,1.000000
